In [1]:
import os
import io
import csv 
from zipfile import ZipFile

In [2]:
#rename "holder" folder

zip_files = os.listdir('bigwedge')

In [3]:
#list of headers

headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

In [5]:
#make a new diretory

#make new directory to hold extracted files 
temp_folder_name = "bigwedge_clean"
if not os.path.isdir(temp_folder_name): #if folder exisits
    os.mkdir(temp_folder_name)          # if not, make it

In [6]:
for zip_file in zip_files : #files found in the wedge directory from the firts extraction of the large zip file
    with ZipFile("bigwedge/" + zip_file, 'r') as my_zip_file:#read the zip files in the directory as my_zip_file
        files_inside = my_zip_file.namelist() #assign new name of files inside 
        
        
        for zipped_file in files_inside :
            sniffer = csv.Sniffer()
            
            
            with my_zip_file.open(zipped_file,'r') as input_file:
                clean_file_name = input_file.name.replace('.csv','_clean.csv')#rename new files 
                
                
                with open("bigwedge_clean/" + clean_file_name,'w') as outfile:#open the output file name in new folder
                    outfile.write(",".join(headers) + "\n") #write the files and join the headers to the new outfile
                    rows_printed = 0
                    
                    
                    for idx,line in enumerate(input_file):
                        file_has_header = False
                        dialect = sniffer.sniff(line.decode("utf-8"))
                        line = line.decode("utf-8").strip().split(dialect.delimiter)#take a bytes object line to a normal string, strip it and then split the string to list
                        line=[piece.replace(r'"','') for piece in line] #use list comprehension to remove the double quotes
                        line=[piece.replace(r"//N",'') for piece in line]#remove null values
                        line=[piece.replace(r"\N",'') for piece in line]
                        line=[piece.replace(r'NULL','') for piece in line]
                        
                        #since we split on commas, the item "Fruit, yogurt" etc split into different columns
                        #we need to combine these columns back into one
                        
                        if len(line) != 50: #our "good" columns are length 50. if > 50, we need to combine
                            new_column = ("".join(line[5:8])) #make a new column by combining the ones that were split by commas
                            #print(new_column)
                            
                            #now remove the bad columns
                            #print(len(line))
                            del(line[5:8]) 
                            #print(line)
                            #print(len(line))
                                
                            #insert the new fixed column
                            line.insert(5,new_column)                            
                            #print(line)
                            #print(len(line))   
                        
                        #if the first row has a datetime in it, then the file has a header
                        if idx == 0:
                            if 'datetime' in line[0]:
                                file_has_header = True
                        if file_has_header and idx == 0:
                            pass #don't print the line
                        else:
                            outfile.write(",".join(line) + "\n")
                            rows_printed += 1
print(f"Yay!!")

Yay!!
